In [5]:
# Import necessary libraries
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph import MessagesState, START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
import keyring

# Define tools
@tool
def play_song_on_spotify(song: str):
    """Doc string"""
    return f"Successfullt played {song} on Spotify!"

@tool
def play_song_on_apple(song: str):
    """Doc string"""
    return f"Successfully played {song} on Apple Music!"

# List of tools
tools = [play_song_on_apple, play_song_on_spotify]
tool_node = ToolNode(tools)

# Set up model
model = ChatOpenAI(model='gpt-4o-mini', api_key=keyring.get_password('openai', 'key_for_windows')).bind_tools(tools)

# Define model-calling function
def call_model(state):
    response = model.invoke(state['messages'])
    return {"messages": [response]}

# Define continuation logic
def should_continue(state):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "continue"
    return "end"

In [6]:
# Build the graph
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# Define graph flow
workflow.add_edge(START, "agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue":"action",
        "end":END
    }
)

# Set up memory for checkpointing
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [7]:
# Interacting with the Agent
from langchain_core.messages import HumanMessage
config = {
    "configurable": {"thread_id": "1"}
}
input_message = HumanMessage(
    content="Can you play Taylor Swift's most popular song?"
)
for event in app.stream({"messages":[input_message]}, config, stream_mode="values"):
    print(event['messages'][-1].pretty_print())

================================ Human Message =================================

Can you play Taylor Swift's most popular song?
None
================================== Ai Message ==================================
Tool Calls:
  play_song_on_apple (call_0dLhVp3pGCikLPRjiWVBECJx)
 Call ID: call_0dLhVp3pGCikLPRjiWVBECJx
  Args:
    song: Blank Space by Taylor Swift
  play_song_on_spotify (call_MYgaLJJJKnZe0hUGI7nDCDsM)
 Call ID: call_MYgaLJJJKnZe0hUGI7nDCDsM
  Args:
    song: Blank Space by Taylor Swift
None
================================= Tool Message =================================
Name: play_song_on_spotify

Successfullt played Blank Space by Taylor Swift on Spotify!
None
